# 텐서플로우 튜토리얼 #03-B
# 레이어 API

by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)
/ [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials) / [Videos on YouTube](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)

## 경고!

**레이어 API는 텐서플로에서 Neural Networks를 생성하기 위한 기본 빌더 API로 의도되었지만 Layers API가 완전히 완성된 적은 없었다. 텐서플로 대 1.9에서는 여전히 작동하지만, 앞으로 더 이상 사용되지 않을 가능성이 꽤 있어 보인다. 대신 보다 완전한 _Keras API_를 사용하는 것이 좋다. 자습서 #03-C를 참조하십시오.**
   

## 소개

텐서플로우(TensorFlow)에서 Neural Networks를 구축할 때는 소스 코드를 구현하고 수정하기 쉽기 때문에 빌더 API를 사용하는 것이 중요하다. 이것은 또한 벌레의 위험을 낮춘다.

다른 튜토리얼 중 상당수가 뉴럴 네트워크의 손쉬운 구축을 위해 PretyTensor라는 텐서플로 빌더 API를 사용했다. 그러나 텐서플로우를 위한 몇 가지 다른 빌더 API가 있다. 이 튜토리얼에는 FeatTensor가 사용되었는데, 당시 2016년 중반에는 FeatTensor가 텐서플로우가 사용할 수 있는 가장 완벽하고 세련된 빌더 API였기 때문이다. 하지만 프리티텐서는 구글에서 일하는 한 사람에 의해서만 개발되며, 비록 독특하고 우아한 특징을 가지고 있지만, 미래에 그것이 더 이상 사용되지 않을 수도 있다.

이 튜토리얼은 최근 텐서플로 버전 1.1에 추가된 소형 빌더 API에 관한 것이다. 단순히 *레이어* 또는 *레이어 API* 또는 파이썬 이름 tf.레이어즈(tf.layers)로 불린다. 이 빌더 API는 텐서플로우의 일부로 자동 설치되기 때문에 더 이상 PyteTensor와 함께 필요했던 대로 별도의 파이썬 패키지를 설치할 필요가 없다.

이 자습서는 Feattensor의 Tutorial #03과 매우 유사하며 Layers API를 사용하여 동일한 Convolutional Neural Network를 구현하는 방법을 보여준다. 콘볼루션 신경 네트워크의 튜토리얼 #02에 익숙할 것을 권장한다.

## 플로우 차트

다음 도표는 아래에 구현된 컨볼루션 신경망에서 데이터가 어떻게 흐르는지 대략적으로 보여준다. 콘볼루션에 대한 자세한 설명은 자습서 #02를 참조하십시오.

![Flowchart](images/02_network_flowchart.png)

입력 이미지는 필터-가중치를 사용하여 첫 번째 경련층에서 처리된다. 이로 인해 16개의 새로운 이미지가 생성되며, 각 필터마다 한 개의 합성 층이 생성된다. 영상 또한 최대 풀을 사용하여 하향 샘플링되므로 영상 해상도가 28x28에서 14x14로 감소한다.

이 16개의 작은 이미지들은 두 번째 경련층에서 처리된다. 우리는 이 16개 채널 각각에 대한 필터-가중치가 필요하고, 이 계층의 각 출력 채널에 대한 필터-가중치가 필요하다. 출력 채널은 36개가 있으므로 두 번째 경련층에는 총 16 x 36 = 576개의 필터가 있다. 결과 영상도 최대 풀링을 사용하여 7x7 픽셀까지 다운샘플링된다.

제2 콘볼루션층의 출력은 각각 7x7픽셀의 36개 이미지다. 그리고 나서 이것들은 길이 7 x 7 x 36 = 1764의 단일 벡터로 평평하게 되는데, 이것은 128개의 뉴런(또는 원소)으로 완전히 연결된 층에 대한 입력으로 사용된다. 이것은 10개의 뉴런으로 완전히 연결된 또 다른 층으로 공급되는데, 각 계층마다 하나씩, 즉 영상의 등급을 결정하는 데 사용되는, 즉 영상에 어떤 숫자가 묘사되어 있는가 하는 것이다.

경련형 필터는 처음에 무작위로 선택되기 때문에 분류는 무작위로 이루어진다. 입력 영상의 예측 클래스와 실제 클래스 사이의 오차는 이른바 교차 엔트로피로 측정된다. 그런 다음 최적기는 분화의 체인 규칙을 사용하여 자동으로 이 오류를 콘볼루션 네트워크를 통해 다시 전파하고 분류 오류를 개선하도록 필터 가중치를 업데이트한다. 이는 분류 오류가 충분히 낮을 때까지 반복적으로 수천 번 이루어진다.

이러한 특정 필터 무게와 중간 이미지는 하나의 최적화 실행의 결과물이며 이 노트북을 다시 실행하면 다르게 보일 수 있다.

TensorFlow의 계산은 실제로 단일 영상 대신 영상 배치에서 수행되므로 계산의 효율성이 향상된다는 점에 유의하십시오. 이는 실제로 TensorFlow에서 구현했을 때 플로우차트가 한 가지 더 데이터 경감을 갖는다는 것을 의미한다. 

## Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import math

이것은 Python 3.6 (Anaconda)과 TensorFlow 버전을 사용하여 개발되었다.

In [2]:
tf.__version__

'1.1.0'

## 데이터 로드

MNIST 데이터 세트는 약 12MB로 주어진 경로에 위치하지 않으면 자동으로 다운로드된다.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

ModuleNotFoundError: No module named 'tensorflow'

MNIST 데이터 세트는 현재 로드되었으며 7만 개의 영상과 관련 라벨(즉, 영상 분류)으로 구성되어 있다. 데이터 세트는 3개의 상호 배타적인 하위 세트로 분할된다. 이번 튜토리얼에서는 훈련과 시험 세트만 사용할 것이라고 말했다.

In [ ]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))

클래스 라벨은 One-Hot 인코딩된 것으로, 각 라벨은 10개의 원소를 가진 벡터라는 뜻이며, 한 요소를 제외하고는 모두 0이다. 이 한 요소의 색인은 클래스 번호, 즉 관련 이미지에 표시된 숫자다. 시험 세트의 정수로도 학급번영자가 필요하니까 지금 계산하는 거야.

In [2]:
data.test.cls = np.argmax(data.test.labels, axis=1)

NameError: name 'np' is not defined

## 데이터 디멘션

데이터 치수는 아래의 소스 코드의 여러 곳에서 사용된다. 이 변수들은 한 번 정의되기 때문에 우리는 아래의 소스 코드 전체에 걸쳐 숫자 대신 이러한 변수를 사용할 수 있다.

In [3]:
# 우리는 MNIST 영상이 각 차원에 28픽셀이라는 것을 알고 있다.
img_size = 28

# 이미지는 이 길이의 1차원 배열로 저장된다.
img_size_flat = img_size * img_size

# 배열을 재구성하는 데 사용되는 이미지의 높이와 폭을 가진 튜플.
img_shape = (img_size, img_size)

# 영상의 색상 채널 수: 그레이 스케일의 경우 1 채널.
num_channels = 1

# 클래스 수, 열 자리마다 한 클래스씩.
num_classes = 10

### 이미지 플롯을 위한 도우미 함수

9개의 영상을 3x3 그리드에 플로팅하고 각 이미지 아래에 참 클래스와 예측 클래스를 작성하는 데 사용되는 기능

In [4]:
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # 3x3 서브플롯으로 피규어를 만든다.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # 플롯 이미지.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # 진실되고 예측된 클래스를 보여라.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        # x축에 클래스를 라벨로 표시한다.
        ax.set_xlabel(xlabel)
        
        # 플롯에서 눈금을 제거하라.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # 여러 플롯을 사용하여 플롯이 올바르게 표시되도록 하십시오.
    # 단일 노트북 셀에.
    plt.show()

### 데이터가 올바른지 보기 위해 몇 개의 이미지를 플롯하십시오.

In [5]:
# 시험 세트에서 첫 번째 영상을 얻으십시오.
images = data.test.images[0:9]

# 그 이미지들에 대한 진정한 수업을 받으세요.
cls_true = data.test.cls[0:9]

# 위의 도우미 함수를 사용하여 이미지와 라벨을 플롯하십시오.
plot_images(images=images, cls_true=cls_true)

NameError: name 'data' is not defined

## 텐서플로우 그래프

텐서플로우의 전체 목적은 파이톤에서 직접 동일한 계산을 수행할 경우보다 훨씬 효율적으로 실행할 수 있는 이른바 계산 그래프를 갖추는 것이다. 텐서플로우는 실행해야 하는 전체 연산 그래프를 알고 있는 반면, NumPy는 한 번에 하나의 수학 연산만을 알고 있기 때문에 텐서플로우는 NumPy보다 효율적일 수 있다.

텐서플로우의 전체 목적은 파이톤에서 직접 동일한 계산을 수행할 경우보다 훨씬 효율적으로 실행할 수 있는 이른바 계산 그래프를 갖추는 것이다. 텐서플로우는 실행해야 하는 전체 연산 그래프를 알고 있는 반면, NumPy는 한 번에 하나의 수학 연산만을 알고 있기 때문에 텐서플로우는 NumPy보다 효율적일 수 있다.

TensorFlow는 GPU뿐만 아니라 멀티 코어 CPU도 활용할 수 있으며 구글은 TPU(Tensor Processing Units)라고 불리며 GPU보다 더 빠른 텐서플로우만을 위한 특수 칩까지 만들었다.

TensorFlow 그래프는 아래에 자세히 설명될 다음과 같은 부분으로 구성된다:

* 그래프로 데이터를 입력하는 데 사용되는 자리 표시자 변수.
* 컨볼루션 네트워크를 더 잘 수행할 수 있도록 최적화될 변수.
* 콘볼루션 신경망의 수학적 공식.
* 변수의 최적화를 안내하는 데 사용할 수 있는 이른바 비용 측정 또는 손실 기능.
* 변수를 업데이트하는 최적화 방법.

또한 TensorFlow 그래프는 본 자습서에서 다루지 않는 TensorBoard를 사용하여 표시되는 데이터 로깅과 같은 다양한 디버깅 문도 포함할 수 있다.

## Placeholder variables

플레이스홀더 변수는 우리가 그래프를 실행할 때마다 변경될 수 있는 텐서플로 계산 그래프의 입력 역할을 한다. 우리는 이것을 자리 표시자 변수 공급이라고 부르는데, 그것은 아래에 더 자세히 설명되어 있다.

먼저 우리는 입력 영상에 대한 자리 표시자 변수를 정의한다. 이를 통해 TensorFlow 그래프에 입력되는 영상을 변경할 수 있다. 이것은 단지 다차원 배열이라는 뜻의 이른바 텐서다. 데이터 타입은 `float32`로, 형태는 `[None, img_size_flat]`로 설정되어 있는데, 여기서 `None`은 각각의 이미지가 길이 `img_size_flat`의 벡터인 상태에서 임의의 수의 영상을 담을 수 있다는 것을 의미한다.

In [6]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

NameError: name 'tf' is not defined

콘볼루션층에서는 `x`가 4차원 텐서로 인코딩될 것으로 예상하므로 그 모양이 `[num_images, img_hight, img_width, num_channels]`가 되도록 재구성해야 한다. `img_hight == img_width==img_size`와 `num_images`는 첫 번째 치수 크기에 대해 -1을 사용하여 자동으로 추론할 수 있다. 따라서 재편성 작업은 다음과 같다.

In [7]:
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

NameError: name 'tf' is not defined

다음에는 자리 표시자 변수 `x`에 입력된 이미지와 관련된 실제 레이블에 대한 자리 표시자 변수가 있다. 이 자리 표시자 변수의 모양은 임의의 수의 레이블을 포함할 수 있다는 뜻의 `[None, num_classes]`이며, 각 레이블은 길이가 10인 `num_classes`의 벡터다.

In [8]:
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

NameError: name 'tf' is not defined

클래스 번호에 대한 자리 표시자 변수도 가질 수 있지만 대신 argmax를 사용해 계산하겠다. 이 연산자는 텐서플로우 연산자이므로 이 시점에서는 아무것도 계산되지 않는다는 점에 유의하십시오.

In [9]:
y_true_cls = tf.argmax(y_true, dimension=1)

NameError: name 'tf' is not defined

## 프리티 텐서 구현

이 절은 Tutorial #03에서 가져온 FeatTensor를 이용한 Convolutional Neural Network의 구현을 보여줌으로써 아래의 Layers API를 이용한 구현과 비교할 수 있다. 이 코드는 if False: 블록으로 동봉되어 있어 여기서 실행되지 않는다.

입력 텐서 `x_image`를 새로운 연산 레이어를 추가하는 도우미 기능이 있는 Feattensor 객체에 포장해 전체 콘볼루션 신경망을 만드는 것이 기본 아이디어다. 이것은 상당히 단순하고 우아한 구문이다.

In [10]:
if False:
    x_pretty = pt.wrap(x_image)

    with pt.defaults_scope(activation_fn=tf.nn.relu):
        y_pred, loss = x_pretty.\
            conv2d(kernel=5, depth=16, name='layer_conv1').\
            max_pool(kernel=2, stride=2).\
            conv2d(kernel=5, depth=36, name='layer_conv2').\
            max_pool(kernel=2, stride=2).\
            flatten().\
            fully_connected(size=128, name='layer_fc1').\
            softmax_classifier(num_classes=num_classes, labels=y_true)

## 레이어 구현

우리는 이제 텐서플로 버전 1.1에 포함된 레이어 API를 이용하여 동일한 컨버전스 뉴럴 네트워크를 구현한다. 다음은 코멘트일 뿐이지만, 이것은 프리티텐서보다 더 많은 코드를 필요로 하는 코드는 다음과 같다.

신경망을 구축하는 과정에서 마지막 층을 지칭하기 위해 `net`변수를 사용한다. 이렇게 하면 실험하려는 경우 코드의 레이어를 쉽게 추가하거나 제거할 수 있다. 우선 우리는 `net`변수를 재구성한 입력 이미지로 설정한다.

In [11]:
net = x_image

NameError: name 'x_image' is not defined

입력 이미지는 첫 번째 경련층에 입력되는데, 이 경련층에는 크기가 5x5픽셀씩 16개의 필터가 있다. 활성화 기능은 Tutorial #02에 자세히 설명된 Corrective Linear Unit(ReLU)이다.

In [12]:
net = tf.layers.conv2d(inputs=net, name='layer_conv1', padding='same',
                       filters=16, kernel_size=5, activation=tf.nn.relu)

NameError: name 'tf' is not defined

이러한 방식으로 신경망을 구축할 때의 장점 중 하나는 이제 쉽게 층에 대한 참조를 끌어낼 수 있다는 겁니다. 이것은 프리티텐서에서는 더 복잡했다.

아래 더 나아가 우리는 첫 번째 경련층의 출력을 그림으로 그리고자 하기 때문에, 우리는 그 층에 대한 참조를 보유하기 위한 또 다른 변수를 만든다.

In [13]:
layer_conv1 = net

NameError: name 'net' is not defined

우리는 이제 콘볼루션 층의 산출물에 대한 최대 풀을 한다. 이는 자습서 #02에서도 자세히 설명하였다.

In [14]:
net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

NameError: name 'tf' is not defined

5x5 픽셀의 필터 각각 36개가 달린 두 번째 콘볼루션 레이어와 다시 ReLU 활성화 기능을 추가했다

In [15]:
net = tf.layers.conv2d(inputs=net, name='layer_conv2', padding='same',
                       filters=36, kernel_size=5, activation=tf.nn.relu)

NameError: name 'tf' is not defined

우리는 또한 이 경련층의 산출물을 그림으로 그리고 싶기 때문에 나중에 사용하기 위해 참조를 유지한다.

In [16]:
layer_conv2 = net

NameError: name 'net' is not defined

두 번째 콘볼루션층의 출력도 영상을 다운샘플링하기 위해 최대 풀로 되어 있다.

In [17]:
net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

NameError: name 'tf' is not defined

이 최대 풀링에 의해 출력되고 있는 텐서는 다음과 같이 4등급이다.

In [18]:
net

NameError: name 'net' is not defined

다음으로는 뉴럴 네트워크에 완전히 연결된 층을 추가하고 싶지만, 이 층들은 입력으로 2등급 텐서가 필요하기 때문에 우선 텐더를 평평하게 해야 한다.

`tf.layers` API는 텐서플로우 코어에 옮겨지기 전 `tf.contrib.layer`에 처음 위치했다. 그러나 텐서플로우 개발자들이 이런 간단한 기능들을 옮기는 데 1년이 걸렸지만 그들은 심지어 더 간단한 `flatten()` 기능을 옮기는 것을 잊어버렸다. 그래서 우리는 `tf.contrib.layers`에 있는 것을 여전히 사용할 필요가 있다.

In [19]:
net = tf.contrib.layers.flatten(net)

# 이것은 결국 다음으로 대체되어야 한다.:
# net = tf.layers.flatten(net)

NameError: name 'tf' is not defined

이것은 이제 데이터를 2단계 텐서(tensor)로 납작하게 만들었다. 여기서 알 수 있듯이.

In [20]:
net

NameError: name 'net' is not defined

이제 우리는 신경망에 완전히 연결된 층을 추가할 수 있다. 이를 레이어 API에서 *dense* 레이어라고 한다.

In [21]:
net = tf.layers.dense(inputs=net, name='layer_fc1',
                      units=128, activation=tf.nn.relu)

NameError: name 'tf' is not defined

입력 영상을 10가지 등급으로 분류하려면 신경망이 필요하다. 그래서 최종 완전연결층에는 `num_classes=10` 출력 뉴런이 있다.

In [22]:
net = tf.layers.dense(inputs=net, name='layer_fc_out',
                      units=num_classes, activation=None)

NameError: name 'tf' is not defined

최종 완전 연결 레이어의 출력을 로짓이라고 부르기도 하기 때문에 그런 이름의 편의 변수를 가지고 있다.

In [23]:
logits = net

NameError: name 'net' is not defined

우리는 소프트맥스 기능을 사용하여 출력이 0과 1 사이에 있도록 `spuash`하고, 따라서 합쳐서 1이 되도록 한다.

In [24]:
y_pred = tf.nn.softmax(logits=logits)

NameError: name 'tf' is not defined

이것은 신경 네트워크가 입력 이미지가 가능한 각 등급의 것이라고 생각하는 가능성을 말해준다. 가장 높은 값을 가진 것이 가장 가능성이 높은 것으로 간주되어 그 지수는 클래스 번호로 간주된다.

In [25]:
y_pred_cls = tf.argmax(y_pred, dimension=1)

NameError: name 'tf' is not defined

우리는 이제 직접 텐서플로 구현에서 많은 복잡한 코드 라인이 필요했던 코드의 몇 줄에 정확히 같은 콘볼루션 신경망을 만들었다.

계층 API는 아마도 FeattyTensor만큼 우아하지는 않지만, 예를 들어 우리가 더 쉽게 중간 계층을 언급할 수 있고, 계층 API를 사용하여 가지와 다중 출력을 가진 신경 네트워크를 구축하는 것도 더 용이하다.

### 최적화해야 하는 손실 함수

입력된 영상을 더 잘 분류하기 위해서는 어떻게든 콘볼루션 신경망의 변수를 바꿔야 한다.

크로스 엔트로피는 분류에 사용되는 성능 측정값이다. 교차-엔트로피는 항상 양의 함수로서 모델의 예측 출력이 원하는 출력과 정확히 일치하면 교차-엔트로피는 0이다. 따라서 최적화의 목적은 교차 엔트로피를 최소화하여 모델의 변수를 변경하여 가능한 한 0에 가깝게 하는 것이다.
,
TensorFlow는 내부에서도 소프트맥스를 계산하기 때문에 로짓 레이어의 값을 사용하는 교차 엔트로피를 계산하는 기능을 갖고 있어 수학적 안정성을 높일 수 있다.

In [26]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=logits)

NameError: name 'tf' is not defined

우리는 이제 각 이미지 분류에 대한 교차 엔트로피를 계산했기 때문에 모델이 각 이미지에서 개별적으로 얼마나 잘 수행되는지 측정할 수 있다. 그러나 교차 엔트로피를 사용하여 모델의 변수의 최적화를 유도하려면 단일 스칼라 값이 필요하므로 모든 이미지 분류에 대한 교차 엔트로피의 평균을 단순히 취한다.

In [27]:
loss = tf.reduce_mean(cross_entropy)

NameError: name 'tf' is not defined

### 최적화 매서드

이제 최소화해야 할 비용측정이 생겼으니, 그러면 최적기를 만들 수 있다. 이 경우 학습 속도가 1e-4인 아담 최적화 도구다.

이 시점에서는 최적화가 수행되지 않는다는 점에 유의하십시오. 사실 아무것도 계산되지 않고, 나중에 실행할 수 있도록 최적화 도구-개체를 텐서플로 그래프에 추가하기만 하면 된다.

In [28]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)

NameError: name 'tf' is not defined

### 분류 정확도

구분 정확도를 계산해야 사용자에게 진행 상황을 보고할 수 있다.

먼저 우리는 예측된 클래스가 각 이미지의 진정한 클래스와 동일한지 여부를 알려주는 술래들의 벡터를 만든다.

In [29]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

NameError: name 'tf' is not defined

분류 정확도는 우선 술집의 벡터를 물에 뜨게 하여 거짓이 0이 되고 참이 1이 되게 한 다음 이 숫자들의 평균을 취함으로써 계산된다.

In [30]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

NameError: name 'tf' is not defined

### 가중치 얻기

아래 더 나아가서 우리는 콘볼루션 층의 무게를 그려보고자 한다. 텐서 플로우 구현에서는 변수를 직접 참조할 수 있도록 변수를 직접 생성했다. 그러나 `tf.layers`와 같은 빌더 API를 사용하여 네트워크를 구축할 때 계층의 모든 변수는 빌더 API에 의해 간접적으로 생성된다. 그러므로 우리는 텐서플로에서 변수를 회수해야 한다.

우선 텐서플로 그래프의 변수 이름 목록이 필요하다.

In [31]:
for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
    print(var)

NameError: name 'tf' is not defined

각각의 콘볼루션층에는 두 가지 변수가 있다. 첫 번째 경련층에는 `layer_conv1/kernel:0`과 `layer_conv1/bias:0`으로 명명된다. `kernel` 변수는 우리가 아래에서 더 나아가서 구상하고자 하는 변수들이다.

새로운 변수를 만들거나 기존 변수를 재사용하는 등 다른 용도로 설계된 텐서플로 함수 `get_variable()`을 사용해야 하기 때문에 이러한 변수에 대한 참조를 얻기가 다소 어색하다. 가장 쉬운 것은 다음과 같은 도우미 함수를 만드는 것이다.

In [32]:
def get_weights_variable(layer_name):
    # 스코프에서 'kernel'이라는 기존 변수 검색
    # 지정된 layer_name과 함께
    # 텐서플로 함수가 있어서 어색하다.
    # 정말로 다른 목적을 위해 의도된

    with tf.variable_scope(layer_name, reuse=True):
        variable = tf.get_variable('kernel')

    return variable

이 도우미 함수를 이용하면 변수를 되찾을 수 있다. 이것들은 텐서플로우 물체 입니다. 변수의 내용을 얻으려면 아래에 설명된 `contents=session.run(weights_conv1)`과 같은 작업을 수행해야 한다.

In [33]:
weights_conv1 = get_weights_variable(layer_name='layer_conv1')
weights_conv2 = get_weights_variable(layer_name='layer_conv2')

NameError: name 'tf' is not defined

## 텐서플로우 실행

### 텐서 플로우 세션 생성

TensorFlow 그래프가 생성되면 그래프를 실행하는 데 사용되는 TensorFlow 세션을 생성해야 한다.

In [34]:
session = tf.Session()

NameError: name 'tf' is not defined

### 변수 초기화

TensorFlow 그래프의 변수는 최적화를 시작하기 전에 초기화되어야 한다.

In [35]:
session.run(tf.global_variables_initializer())

NameError: name 'session' is not defined

## 최적화 반복을 수행하는 도우미 함수

훈련 세트에는 5만 5천 개의 이미지가 있다. 이 모든 영상을 사용하여 모델의 그라데이션 계산에 오랜 시간이 걸린다. 따라서 최적기의 각 반복에는 작은 이미지 배치만 사용한다.

RAM이 부족하여 컴퓨터가 고장 나거나 속도가 매우 느려지면 이 숫자를 줄이려고 할 수 있지만 최적화 반복을 더 많이 수행해야 할 수도 있다. 

In [36]:
train_batch_size = 64

이 함수는 신경망 층의 변수를 점진적으로 개선하기 위해 다수의 최적화 반복을 수행한다. 각 반복에서 교육 세트에서 새로운 데이터 배치를 선택한 다음 텐서플로우는 해당 교육 샘플을 사용하여 최적기를 실행한다. 진행상황은 100회마다 출력된다.

In [37]:
# 지금까지 수행된 총 반복 횟수에 대한 카운터
total_iterations = 0

def optimize(num_iterations):
    # 로컬 복사본이 아닌 글로벌 변수를 업데이트하십시오.
    global total_iterations

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        # 교육용 예시 한 묶음 가져오기
        # x_batch는 이제 한 묶음의 이미지를 가지고 있으며
        # y_true_batch는 해당 이미지의 실제 레이블이다.
        x_batch, y_true_batch = data.train.next_batch(train_batch_size)

        # 적절한 이름을 가진 명령어에 배치
        # TensorFlow 그래프에서 자리 표시자 변수의 경우.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        # 이 교육 데이터 배치를 사용하여 최적화 도구 실행.
        # TensorFlow는 feed_dict_train에 변수를 할당한다.
        # 자리 표시자 변수로 이동한 다음 최적화 도구를 실행하십시오.
        session.run(optimizer, feed_dict=feed_dict_train)

        # 100번 반복마다 상태 출력
        if i % 100 == 0:
            # 교육 세트의 정확도 계산.
            acc = session.run(accuracy, feed_dict=feed_dict_train)

            # 출력용 메시지
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"

            # 출력
            print(msg.format(i + 1, acc))

    # 수행된 총 반복 횟수 업데이트
    total_iterations += num_iterations

## 예시 오류를 플롯하는 도우미 함수

잘못 분류된 테스트 세트의 이미지 예시를 플로팅하는 함수.

In [38]:
def plot_example_errors(cls_pred, correct):
    # 이 함수는 아래의 print_test_정확도()에서 호출된다.

    # cls_pred는 다음에 대해 예측된 클래스 번호의 배열이다.
    # 테스트 세트의 모든 이미지

    # correct는 예측 클래스의 여부를 나타내는 부울 배열임
    # 테스트 세트의 각 이미지에 대한 True 클래스와 동일함.

    # 부울 배열 취소
    incorrect = (correct == False)
    
    # 테스트 세트에서 이미지의 이미지를 가져오십시오.
    # 잘못 분류된
    images = data.test.images[incorrect]
    
    # 해당 이미지의 예측 클래스 가져오기
    cls_pred = cls_pred[incorrect]

    # 해당 이미지의 실제 클래스 가져오기
    cls_true = data.test.cls[incorrect]
    
    # 처음 9개 이미지 플롯
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

### 혼동 행렬을 표시하는 도우미 함수

In [39]:
def plot_confusion_matrix(cls_pred):
    # 이 함수는 아래의 print_test_정확도()에서 호출된다.

    # cls_pred는 다음에 대해 예측된 클래스 번호의 배열이다.
    # 테스트 세트의 모든 이미지

    # 테스트 세트에 대한 실제 분류 가져오기
    cls_true = data.test.cls
    
    # sklearn을 사용하여 혼동 매트릭스 가져오기
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    # 혼동 행렬을 텍스트로 출력
    print(cm)

    # 혼동 행렬을 이미지로 표시
    plt.matshow(cm)

    # 플롯을 다양하게 조정
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # 그림이 여러 그림으로 올바르게 표시되는지 확인
    # 단일 노트북 셀에.
    plt.show()

### 성능을 보여주는 도우미 함수

아래는 시험 세트에 분류 정확도를 인쇄하는 함수이다.

테스트셋의 모든 영상에 대한 분류를 계산하는 데 시간이 걸리므로, 이 함수에서 위의 함수를 직접 호출하여 결과를 다시 사용하는 것이므로 각 함수에 의해 분류를 다시 계산할 필요가 없다.

이 함수는 컴퓨터 메모리를 많이 사용할 수 있으므로 테스트 세트가 더 작은 배치로 분할되는 것에 유의하십시오. 컴퓨터에 RAM이 거의 없는데 작동이 안 되면 배치 크기를 줄이려고 하면 된다.

In [40]:
# 테스트 세트를 이 크기의 작은 배치로 분할
test_batch_size = 256

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    # 테스트 세트의 이미지 수
    num_test = len(data.test.images)

    # 다음과 같은 예측 클래스에 배열 할당
    # 일괄적으로 계산되어 이 배열로 채워질 것이다.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # 이제 배치에 대한 예측 클래스를 계산하십시오.
    # 우리는 단지 모든 배치들을 반복할 것이다.
    # 이것을 하는 더 영리하고 피톤적인 방법이 있을 것이다.

    # 다음 배치의 시작 지수는 i로 표시된다.
    i = 0

    while i < num_test:
        # 다음 배치에 대한 끝 지수는 j로 표시된다.
        j = min(i + test_batch_size, num_test)

        # 인덱스 i와 j 사이의 테스트 세트에서 이미지 가져오기.
        images = data.test.images[i:j, :]

        # 연결된 레이블 가져오기.
        labels = data.test.labels[i:j, :]

        # 이러한 이미지 및 레이블로 피드 딕트 만들기.
        feed_dict = {x: images,
                     y_true: labels}

        #  텐서플로우를 사용하여 예측 클래스 계산.
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        # 다음 배치에 대한 시작 인덱스를 다음으로 설정
        # 현재 배치에 대한 엔드 인덱스
        i = j

    # 테스트 세트의 실제 클래스 번호에 대한 편의 변수
    cls_true = data.test.cls

    # 각 이미지가 올바르게 분류되었는지 여부를 나타내는 부울 배열 만들기
    correct = (cls_true == cls_pred)

    # 올바르게 분류된 이미지 수 계산
    # 부울 배열을 합할 때 False는 0을 의미하고 True는 1을 의미한다.
    correct_sum = correct.sum()

    # 분류 정확도는 정확하게 분류된 수입니다
    # 이미지를 테스트 세트의 총 이미지 수로 나눈다.
    acc = float(correct_sum) / num_test

    # 정확도 출력
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

    # 필요한 경우 일부 잘못된 분류 예제 그림 그리기.
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    # 원하는 경우 혼동 행렬 그림 그리기
    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

## 최적화 전 성능

신경망 변수가 초기화됐을 뿐 전혀 최적화되지 않아 무작위로 영상을 분류할 뿐이어서 시험 세트의 정확도가 매우 낮다.

In [41]:
print_test_accuracy()

NameError: name 'data' is not defined

## 최적화 1번 반복 후 성능

최적기에 대한 학습 비율이 매우 낮게 설정되어 있기 때문에 분류 정확도는 한 번의 최적화 반복만으로 크게 개선되지 않는다.

In [42]:
optimize(num_iterations=1)

NameError: name 'data' is not defined

In [43]:
print_test_accuracy()

NameError: name 'data' is not defined

## 최적화를 100회 반복한 후의 성능

100회의 최적화 반복을 거쳐, 그 모델은 분류 정확도를 현저히 향상시켰다.

In [44]:
%%time
optimize(num_iterations=99) # We already performed 1 iteration above.

NameError: name 'data' is not defined

In [45]:
print_test_accuracy(show_example_errors=True)

NameError: name 'data' is not defined

## 1000번의 최적화 반복 후 성능

1000회 최적화 반복 후 이 모델은 시험 세트의 정확도를 90% 이상으로 크게 높였다.

In [46]:
%%time
optimize(num_iterations=900) # 위에서 100번 반복했다.

NameError: name 'data' is not defined

In [47]:
print_test_accuracy(show_example_errors=True)

NameError: name 'data' is not defined

## 최적화 반복 10,000회 이후 성능

최적화 1만 번 반복한 후, 이 모델은 약 99%의 시험 세트에 대한 분류 정확도를 갖는다.

In [48]:
%%time
optimize(num_iterations=9000) # We performed 1000 iterations above.

NameError: name 'data' is not defined

In [49]:
print_test_accuracy(show_example_errors=True,
                    show_confusion_matrix=True)

NameError: name 'data' is not defined

## 가중치와 층의 시각화

###콘볼루션 가중치 플로팅을 위한 도우미 함수

In [50]:
def plot_conv_weights(weights, input_channel=0):
    # 가중치가 4-dim 변수에 대한 텐서 흐름 ops라고 가정
    # 예: weights_conv1 또는 weights_conv2.\n",
    
    # TensorFlow에서 가중치 변수 값 검색.\n",
    # 아무것도 계산되지 않기 때문에 피드 딕트가 필요하지 않다.\n",
    w = session.run(weights)

    # 가중치에 대한 가장 낮은 값과 가장 높은 값 가져오기
    # 이것은 전체에서 색 강도를 보정하는 데 사용된다
    # 서로 비교될 수 있도록 하는 이미지들
    w_min = np.min(w)
    w_max = np.max(w)

    # Conv. 층에 사용된 필터 수
    num_filters = w.shape[3]

    # 플롯할 그리드 수
    # 필터 수의 반올림, 제곱근
    num_grids = math.ceil(math.sqrt(num_filters))
    
    # 하위 그림 그리드를 사용하여 그림 작성
    fig, axes = plt.subplots(num_grids, num_grids)

    # 모든 필터-가중치 그림 그리기
    for i, ax in enumerate(axes.flat):
        # 유효한 필터-가중치만 표시
        if i<num_filters:
            # 입력 채널의 i번째 필터에 대한 가중치 가져오기
            # 형식에 대한 자세한 내용은 new_conv_layer()를 참조하십시오.
            # 4-dim 텐서의
            img = w[:, :, input_channel, i]

            # 플롯 이미지
            ax.imshow(img, vmin=w_min, vmax=w_max,
                      interpolation='nearest', cmap='seismic')
        
        # 플롯에서 눈금 제거
        ax.set_xticks([])
        ax.set_yticks([])
    
    # 그림이 여러 그림으로 올바르게 표시되는지 확인
    # 단일 노트북 셀에
    plt.show()

### 콘볼루션 층의 출력을 플로팅하기 위한 도우미 함수

In [51]:
def plot_conv_layer(layer, image):
    # 레이어가 4-dim 텐서를 출력하는 텐서플로우 op이라고 가정하십시오
    # 콘볼루션 층의 결과물,
    # 예: layer_layer1 또는 layer_layer2.

    # 하나의 이미지만 포함하는 피드 딕트 생성.
    # y_true는 y_true이기 때문에 공급할 필요가 없다는 점에 유의하십시오
    # 이 계산에 사용되지 않는
    feed_dict = {x: [image]}

    # 레이어의 출력 값 계산 및 검색
    # 그 이미지를 입력할 때
    values = session.run(layer, feed_dict=feed_dict)

    # Conv.층에 사용된 필터 수
    num_filters = values.shape[3]

    # 플롯할 그리드 수
    # 필터 수의 반올림, 제곱근
    num_grids = math.ceil(math.sqrt(num_filters))
    
    # 하위 그림 그리드를 사용하여 그림 작성
    fig, axes = plt.subplots(num_grids, num_grids)

    # 모든 필터의 출력 영상 플롯
    for i, ax in enumerate(axes.flat):
        # 유효한 필터에 대한 영상만 표시
        if i<num_filters:
            # Get the output image of using the i'th filter.
            img = values[0, :, :, i]

            # 플롯 이미지
            ax.imshow(img, interpolation='nearest', cmap='binary')
        
        # 플롯에서 눈금 제거
        ax.set_xticks([])
        ax.set_yticks([])
    
    # 그림이 여러 그림으로 올바르게 표시되는지 확인
    # 단일 노트북 셀에
    plt.show()

### 입력 이미지

이미지를 플롯하는 데 도움이 되는 함수

In [52]:
def plot_image(image):
    plt.imshow(image.reshape(img_shape),
               interpolation='nearest',
               cmap='binary')

    plt.show()

아래 예제로 사용할 테스트 세트의 이미지를 플롯하십시오.

In [53]:
image1 = data.test.images[0]
plot_image(image1)

NameError: name 'data' is not defined

테스트 세트에서 다른 예제 이미지를 플롯하십시오.

In [54]:
image2 = data.test.images[13]
plot_image(image2)

NameError: name 'data' is not defined

### 콘볼루션 층 1

이제 첫 번째 경련 층에 대한 필터-가중치를 플롯하십시오.

양체중은 빨간색이고 음체중은 파란색이라는 점에 유의하십시오.

In [55]:
plot_conv_weights(weights=weights_conv1)

NameError: name 'weights_conv1' is not defined

이러한 각 콘볼루션 필터를 첫 번째 입력 영상에 적용하면 다음과 같은 출력 영상이 제공되며, 이는 두 번째 콘볼루션층에 대한 입력으로 사용된다.

In [56]:
plot_conv_layer(layer=layer_conv1, image=image1)

NameError: name 'layer_conv1' is not defined

다음 이미지는 두 번째 이미지에 콘볼루션 필터를 적용한 결과 입니다.

In [57]:
plot_conv_layer(layer=layer_conv1, image=image2)

NameError: name 'layer_conv1' is not defined

### 콘볼루션 층 2

이제 두 번째 콘볼루션층에 대한 필터-가중치를 그려보십시오.

첫 번째 콘볼루션층에서 16개의 출력 채널이 있는데, 이것은 두 번째 콘볼루션층으로 16개의 입력 채널이 있다는 것을 의미한다. 두 번째 콘블레이어에는 각 입력 채널에 대한 필터-가중치 집합이 있다. 첫 번째 채널의 필터-가중치를 계획하는 것부터 시작합시다.

다시 한 번 긍정적인 체중은 빨간색이고 부정적인 체중은 파란색이라는 것을 주목하라.

In [58]:
plot_conv_weights(weights=weights_conv2, input_channel=0)

NameError: name 'weights_conv2' is not defined

제2의 콘볼루션층에는 16개의 입력 채널이 있으므로 이와 같은 필터-가중치를 15개 더 만들 수 있다. 두 번째 채널의 필터-가중치로 한 개만 더 만들면 된다.

In [59]:
plot_conv_weights(weights=weights_conv2, input_channel=1)

NameError: name 'weights_conv2' is not defined

차원이 높기 때문에 이러한 필터가 어떻게 적용되는지 이해하고 추적하기가 어려울 수 있다.

이러한 콘볼루션 필터를 첫 번째 콘블레이어로부터 출력된 영상에 적용하면 다음과 같은 영상이 나온다.

이러한 것들은 원래 입력 이미지의 절반인 14 x 14 픽셀로 다운샘플링된다는 점에 유의하십시오. 첫 번째 경구층에는 보폭을 가진 최대 풀링 층이 뒤따랐기 때문이다.2. 최대 풀링 또한 두 번째 경구층 이후에 이루어지지만, 우리는 그 전에 이러한 이미지들을 회수한다.

In [60]:
plot_conv_layer(layer=layer_conv2, image=image1)

NameError: name 'layer_conv2' is not defined

그리고 두 번째 이미지에 필터-가중치를 적용한 결과 입니다.

In [61]:
plot_conv_layer(layer=layer_conv2, image=image2)

NameError: name 'layer_conv2' is not defined

### 텐서플로우 세션 닫기

이제 텐서플로우를 이용한 작업이 끝났기 때문에 그 자원을 공개하기 위해 세션을 닫는다.

In [62]:
# 수정 및 실험을 원할 경우에 대비하여 언급된 사항.
# 노트북을 다시 시작할 필요 없이.
# session.close()

## 결론

이번 튜토리얼에서는 텐서플로우에서 콘볼루션 신경망을 쉽게 구축할 수 있도록 이른바 *레이어스 API*를 사용하는 방법을 보여 주었다. 구문은 프리티텐서보다 다르고 장황하다. 빌더 API는 둘 다 장단점이 있지만 프리티텐서는 한 사람만 개발하고 레이어즈 API는 현재 텐서플로 코어의 공식 부분이기 때문에 향후 프리티텐서가 더 이상 사용되지 않을 가능성도 있다. 이렇게 되면 그 독특하고 우아한 특징들 중 일부가 텐서플로우 코어에도 통합되기를 바랄 수 있을 것이다.

1년 가까이 텐서플로우 개발자로부터 명확한 답을 얻으려고 노력했는데, 그 중 어떤 API가 텐서플로우의 메인 빌더 API가 될 것인가. 아직 미정이고 실행이 매우 더딘 것 같다.

## 연습

텐서플로우(TensorFlow)로 여러분의 실력을 향상시키는 데 도움이 될 수 있는 몇 가지 운동 제안사항들이다. 텐서플로우를 제대로 사용하는 방법을 익히기 위해서는 텐서플,
변경하기 전에 이 노트북을 백업하십시오.

* 일부 레이어에 대해 활성화 기능을 sigmoid로 변경하십시오.
* 모든 계층의 활성화 기능을 간단히 변경할 수 있는 방법을 찾을 수 있는가?
* 완전히 연결된 레이어 다음에 드롭아웃 레이어를 추가한다. 훈련과 테스트 중에 다른 확률을 원하는 경우 자리 표시자 변수가 필요하며 피드 딕트에 설정하십시오.
* conv-layer 대신 최대 풀링 레이어의 출력을 플롯하십시오.
* 2x2 최대 풀링 레이어를 경련 레이어의 stride=2로 교체한다. 분류 정확도에 차이가 있는가? 반복해서 최적화하면 어떨까? 차이가 무작위인데 정말 차이가 있다면 어떻게 측정하시겠습니까? 수녀원에서 최대 풀링 대 보폭을 사용하는 것의 장단점은 무엇인가?
* 커널, 깊이, 크기 등 레이어에 대한 파라미터를 변경한다. 시간 사용과 분류 정확도의 차이는 무엇인가?
* 일부 콘볼레이션 및 완전 연결성 레이어를 추가 및 제거하십시오.
* 여전히 성능이 좋은 가장 단순한 네트워크는 무엇인가?
* 콘볼루션층에 대한 바이어스 값을 검색하여 인쇄하십시오. 영감을 얻으려면 `get_weights_variable()`을 참조하십시오.
* 이 소스 코드를 너무 많이 보지 말고 직접 프로그램을 리메이크하십시오.
* 그 프로그램이 어떻게 작동하는지 친구에게 설명하라.

## 라이센스(MIT)

저작권 (c) 2016 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)

이 소프트웨어의 사용, 복사, 수정, 병합, 게시, 배포, 하위 라이선스 및/또는 판매 권한을 포함하여 제한 없이 본 소프트웨어 및 관련 문서 파일(\"Software\")의 사본을 입수하는 모든 사용자에게 무료로 권한을 부여한다.다음과 같은 조건에 따라 소프트웨어가 제공될 수 있는 이들:

위의 저작권 고지 및 이 허가 고지는 소프트웨어의 모든 사본 또는 상당 부분에 포함되어야 한다.

상품성, 특정 목적에의 적합성 및 비침해의 보증을 포함하되 이에 국한되지 않는 모든 종류의 명시적 또는 묵시적 보증 없이 \"있는 그대로 \" 소프트웨어는 제공된다. 어떠한 경우에도 저작자나 저작권 소유자는 책임을 지지 않는다. 그렇지 않으면 소프트웨어, 소프트웨어 사용 또는 기타 거래에서 발생하거나, 소프트웨어와 연결되거나, 연결된다.